# IMPORTING LIBRARIES

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sklearn.metrics.pairwise import sigmoid_kernel
import pickle

# 1. LOADING DATA

In [2]:
collaborative = pd.read_csv('collaborative.csv')

In [3]:
collaborative

,movieId,46,229,231,332,429,483,533,567,682,...,270377,270414,270524,270551,270555,270564,270579,270634,270654,270887
0,1,4.5,3.0,2.5,4.5,4.0,3.5,3.0,2.0,4.0,...,4.5,5.0,4.0,1.5,3.5,0.0,0.0,4.0,3.0,5.0
1,10,0.0,0.0,3.0,0.0,2.5,3.5,4.0,0.0,3.0,...,0.0,2.5,4.0,3.5,3.0,3.0,3.0,0.0,0.0,0.0
2,11,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,...,0.0,0.0,2.5,0.0,3.0,0.0,3.0,0.0,4.0,4.0
3,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0
4,15,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4582,172853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4583,173857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4584,174533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4585,174591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
content = pd.read_csv('content_based.csv')

In [5]:
content

,tmdb_id,imdb_id_x,movie_id,movie_title,data
0,19995.0,499549.0,72998.0,Avatar,Avatar Action Adventure Fantasy Sci-Fi avatar...
1,285.0,449088.0,53125.0,Pirates of the Caribbean: At World's End,Pirates of the Caribbean: At World's End Acti...
2,206647.0,2379713.0,136020.0,Spectre,Spectre Action Adventure Thriller bomb espion...
3,49026.0,1345836.0,91529.0,The Dark Knight Rises,The Dark Knight Rises Action Thriller decepti...
4,140607.0,5289954.0,NaN,Star Wars: Episode VII - The Force Awakens ...,Star Wars: Episode VII - The Force Awakens ...
...,...,...,...,...,...
4956,72766.0,1880418.0,91829.0,Newlyweds,Newlyweds Comedy Drama written and directed b...
4957,231617.0,3000844.0,164791.0,Signed Sealed Delivered,Signed Sealed Delivered Comedy Drama fraud po...
4958,365441.0,2107644.0,NaN,A Plague So Pleasant,A Plague So Pleasant Drama Horror Thriller na...
4959,126186.0,2070597.0,101294.0,Shanghai Calling,Shanghai Calling Comedy Drama Romance nan Dan...


# COLLABORATIVE

In [6]:
# MAKE CSR MATRIX
collaborative_matrix = csr_matrix(collaborative.drop('movieId', axis=1).values)

In [7]:
# MAKING KNN MODEL
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute').fit(collaborative_matrix)

In [8]:
def collaborative_rec():
    # GET RANDOM MOVIE
    query_index = np.random.choice(collaborative['movieId'])
    print(content.loc[content['movie_id'] == query_index, 'movie_title'].values)
    # FIND NEAREST MOVIES
    distances, indices = model_knn.kneighbors(collaborative[collaborative['movieId'] == query_index].drop('movieId', axis=1).values.reshape(1, -1), n_neighbors = 10)
    indices = list(indices[0])
    movie_id = collaborative.iloc[indices]['movieId'].values
    print(content[content['movie_id'].isin(movie_id)]['movie_title'].values)

In [11]:
collaborative_rec()

['Beyond the Valley of the Dolls\xa0']
['Day of the Dead\xa0' 'The Texas Chain Saw Massacre\xa0'
 'The Return of the Living Dead\xa0' 'Videodrome\xa0'
 'Night of the Living Dead\xa0' 'The Return of the Living Dead\xa0'
 'Day of the Dead\xa0' 'Repo Man\xa0' 'Beyond the Valley of the Dolls\xa0'
 "A Hard Day's Night\xa0" 'The Texas Chain Saw Massacre\xa0'
 'Night of the Living Dead\xa0' 'Eraserhead\xa0' 'Pink Flamingos\xa0']


In [12]:
# SAVE MODEL
filename = 'KNN.sav'
pickle.dump(model_knn, open(filename, 'wb'))

In [13]:
# LOAD MODEL
KNN = pickle.load(open(filename, 'rb'))
query_index = np.random.choice(collaborative['movieId'])
print(content.loc[content['movie_id'] == query_index, 'movie_title'].values)
distances, indices = KNN.kneighbors(collaborative[collaborative['movieId'] == query_index].drop('movieId', axis=1).values.reshape(1, -1), n_neighbors = 10)
indices = list(indices[0])
indices = collaborative.iloc[indices]['movieId'].values
print(content[content['movie_id'].isin(indices)]['movie_title'].values)

['The Amazing Spider-Man\xa0']
['The Dark Knight Rises\xa0' 'Man of Steel\xa0' 'The Avengers\xa0'
 'The Amazing Spider-Man\xa0' 'Iron Man 3\xa0'
 'X-Men: Days of Future Past\xa0'
 'Captain America: The Winter Soldier\xa0' 'X-Men: First Class\xa0'
 'Thor\xa0' 'Captain America: The First Avenger\xa0' 'The Avengers\xa0']


# CONTENT

In [55]:
content = content.drop_duplicates(subset='movie_title', keep="last")

In [56]:
# COMPUTE TFIDF 
v = TfidfVectorizer()
x = v.fit_transform(content['data'])

In [57]:
# SIGMOID KERNEL
sig = sigmoid_kernel(x, x)

In [58]:
indices = pd.Series(content.index, index=content['tmdb_id'].apply(lambda x: int(x))).drop_duplicates()
rev = pd.Series(content['tmdb_id'].apply(lambda x: int(x)), index=content.index).drop_duplicates()

In [59]:
def content_rec():
    # GET TMDB ID
    tmdb = int(np.random.choice(content['tmdb_id']))
    print(content.loc[content['tmdb_id'] == tmdb, 'movie_title'].values)
    idx = indices[tmdb]
    
    # GET PAIRWISE SIMILARITY SCORE 
    sig_scores = list(enumerate(sig[idx]))

    # SORTING MOVIES
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # GET MOST SIMILAR MOVIES PAIRS
    sig_scores = sig_scores[1:15]

    # GET TMDB ID
    idx = [i[0] for i in sig_scores]
    tmdb = []
    for i in idx:
        try:
            tmdb.append(rev[i])
        except:
            pass

    # PRINT SIMILAR MOVIES
    print([content[content['tmdb_id'] == float(i)]['movie_title'].values[0] for i in tmdb])

In [61]:
content_rec()

['Don Juan DeMarco\xa0']
['Ruby in Paradise\xa0', 'Nacho Libre\xa0', 'Fear Clinic\xa0', 'The Road to El Dorado\xa0', 'The Musketeer\xa0', 'I Got the Hook Up\xa0', 'Elizabeth\xa0', 'The Barbarian Invasions\xa0', 'Nancy Drew\xa0', 'Girl, Interrupted\xa0', "The Devil's Tomb\xa0", 'State of Play\xa0', '200 Cigarettes\xa0']


In [62]:
# SAVE MODEL
filename = 'SIG.sav'
pickle.dump(sig, open(filename, 'wb'))

In [63]:
# LOAD MODEL

tmdb = int(np.random.choice(content['tmdb_id']))
print(content.loc[content['tmdb_id'] == tmdb, 'movie_title'].values)
idx = indices[tmdb]
sig_scores = list(enumerate(SIG[idx]))
sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
sig_scores = sig_scores[1:15]
idx = [i[0] for i in sig_scores]
tmdb = []
for i in idx:
    try:
        tmdb.append(rev[i])
    except:
        pass
print([content[content['tmdb_id'] == float(i)]['movie_title'].values[0] for i in tmdb])

['Saints and Soldiers\xa0']
["Hart's War\xa0", 'An Unfinished Life\xa0', 'The Celebration\xa0', 'Jeff, Who Lives at Home\xa0', 'Wag the Dog\xa0', 'Bucky Larson: Born to Be a Star\xa0', 'Scary Movie 5\xa0', "Mr. Bean's Holiday\xa0", "Monster's Ball\xa0", 'O\xa0', 'Touching the Void\xa0', 'Moon\xa0', '127 Hours\xa0', '8 Mile\xa0']
